# 02 — LLM Triage & Explainability (Azure OpenAI Responses API)
Consumes Vision proposals and performs multimodal triage; returns structured JSON.

**Prerequisites**
- Azure OpenAI resource
- Python SDK: openai (Azure)
- Image URL (SAS)


In [ ]:
# !pip install openai
from openai import AzureOpenAI
import os, json

AOAI_ENDPOINT = os.getenv('AOAI_ENDPOINT','https://<your-aoai>.openai.azure.com/')
AOAI_KEY = os.getenv('AOAI_KEY','<your-key>')
MODEL = os.getenv('AOAI_MODEL','gpt-4.1')
IMAGE_URL = os.getenv('IMAGE_URL','<sas-or-https-url>')

with open('artifacts/proposals.json','r') as f:
    region_props = json.load(f)

SYSTEM_PROMPT = 'Paste the multi-defect System Prompt here.'

client = AzureOpenAI(azure_endpoint=AOAI_ENDPOINT, api_key=AOAI_KEY)
response = client.responses.create(
  model=MODEL,
  input=[
    {'role':'system','content':SYSTEM_PROMPT},
    {'role':'user','content':[
        {'type':'input_text','text':'Analyze proposals and return JSON.'},
        {'type':'input_image','image_url':{'url': IMAGE_URL}},
        {'type':'input_text','text': json.dumps(region_props)}
    ]}
  ],
  response_format={'type':'json_object'}
)
output_json = response.output[0].content[0].text
print(output_json)
os.makedirs('artifacts', exist_ok=True)
with open('artifacts/llm_triage.json','w') as f:
    f.write(output_json)
print('Saved artifacts/llm_triage.json')
